In [74]:
import pandas as pd
from ics import Calendar, Event, DisplayAlarm
from datetime import datetime, timedelta

# read scraped output
df = pd.read_csv("output.csv", names=["url","artist_country","artist","location","date_string","insta","spotify"])

# read ratings and comments from google sheet
SHEET_ID = '1uu3_S_e2zR5O2cTbknIUh4bG1GymBtdwieFwZynTPRY'
SHEET_NAME = 'transform'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
df_ratings = pd.read_csv(url,usecols=['🧑‍🎤 artist 👩‍🎤','⭐ Mikael rating','⭐ Line rating','💭 comment'])

# rename
df_ratings = df_ratings.rename(columns={'🧑‍🎤 artist 👩‍🎤':'artist','⭐ Mikael rating':'m_rating','⭐ Line rating':'l_rating','💭 comment':'comment'})
df_ratings['comment'] = df_ratings['comment'].fillna('')

In [75]:
# join website info with ratings  
df = df.join(df_ratings, how='outer', lsuffix='_left', rsuffix='_right')


In [78]:
# get starttime, fill nulls with timestamp 13:37
df['start_time'] = df['date_string'].str.extract(r'(\d{2}:\d{2})').fillna('13:37')


# strip irrelevant locale information; could probably be made redundant
df['new_date_string'] = df['date_string'].replace(to_replace=r'.*dag (\d{1,2})\. (\w{3})\w', value=r'\1\2', regex=True)

# convert to timestamp
df['start_datetime'] = pd.to_datetime(df['new_date_string'] + df['start_time'], format='%d%b %Y%H:%M').dt.tz_localize('CET')

#ics package needs utc input

## todo handle time of day for ical. Example below
df[['date_string','start_datetime']][135:].head()

,date_string,start_datetime
135,Lørdag 1. juli 2023,2023-07-01 13:37:00+02:00
136,Lørdag 1. juli 2023,2023-07-01 13:37:00+02:00
137,21:45 Lørdag 1. juli 2023,2023-07-01 21:45:00+02:00
138,Lørdag 1. juli 2023,2023-07-01 13:37:00+02:00
139,Lørdag 1. juli 2023,2023-07-01 13:37:00+02:00


In [79]:
df.head()

,url,artist_country,artist_left,location,date_string,insta,spotify,artist_right,m_rating,l_rating,comment,start_time,new_date_string,start_datetime
0,https://www.roskilde-festival.dk/da/years/2023...,US,Kendrick Lamar,Orange,Onsdag 28. juni 2023,https://www.instagram.com/kendricklamar/,https://open.spotify.com/artist/2YZyLoL8N0Wb9x...,Kendrick Lamar,🤯JA!,NaN,,13:37,28jun 2023,2023-06-28 13:37:00+02:00
1,https://www.roskilde-festival.dk/da/years/2023...,US,Queens Of The Stone Age,Arena,Onsdag 28. juni 2023,https://www.instagram.com/queensofthestoneage,https://open.spotify.com/artist/4pejUc4iciQfgd...,Queens Of The Stone Age,😌Tømmermændsmusik,NaN,,13:37,28jun 2023,2023-06-28 13:37:00+02:00
2,https://www.roskilde-festival.dk/da/years/2023...,DK,Blæst,Orange,Onsdag 28. juni 2023,https://www.instagram.com/blaestband/,https://open.spotify.com/artist/3elCsuJ1JjDeAd...,Fever Ray,😌Tømmermændsmusik,NaN,,13:37,28jun 2023,2023-06-28 13:37:00+02:00
3,https://www.roskilde-festival.dk/da/years/2023...,SE,Fever Ray,Arena,Onsdag 28. juni 2023,https://www.instagram.com/feverray/,https://open.spotify.com/artist/5hE6NCoobhyEu6...,Rema,👍Ja,NaN,commercial african pop. So smooth and well-pro...,13:37,28jun 2023,2023-06-28 13:37:00+02:00
4,https://www.roskilde-festival.dk/da/years/2023...,US,Lorna Shore,Avalon,Onsdag 28. juni 2023,https://www.instagram.com/lornashore/?hl=da,https://open.spotify.com/artist/6vXYoy8ouRVib3...,Armand Hammer,😌Tømmermændsmusik,NaN,chill jazz akkorder + rap,13:37,28jun 2023,2023-06-28 13:37:00+02:00


In [81]:
# write most important concerts to ical
c = Calendar()
for index, row in df[df['m_rating'] == '🤯JA!'].iterrows():
    e = Event()
    e.name = row['artist_left']
    e.begin = row['start_datetime']
    e.end = row['start_datetime'] + timedelta(hours=1) # assume all concerts are 1 hour
    e.description = row['comment']
    e.location = row['location']
    a=DisplayAlarm()
    a.trigger=timedelta(hours=-1)
    a.display_text=" concert alert!"
    e.alarms=[a]
    c.events.add(e)

# c.events
with open('most_important.ics', 'w') as my_file:
    my_file.writelines(c.serialize_iter())


/Users/mikaelthorup/repos/RF23/venv/lib/python3.10/site-packages/ics/component.py:85: FutureWarning: Behaviour of str(Component) will change in version 0.9 to only return a short description, NOT the ics representation. Use the explicit Component.serialize() to get the ics representation.
  warnings.warn(
